 # projeto sem objetivo pre definido

* primeiro fazer coleta dos dados para armazenamento 
*  sera feito coleta atraves de web scraping
* site: https://www.zapimoveis.com.br
* limpeza de dados
* talvez crie um banco de dados para mostrar a criacao de um banco de dados
* talvez pode ser feito uma coleta diaria durante um tempo para saber qto tempo um imovel fica disponivel para venda, qual o tempo medio de espera para ser vendido, qual o preço medio dos  imoveis mais vendidos 

apos coleta sera feita uma analise exploratoria para saber o que sera possivel de analisar, como por exemplo, valor media das casa, valor medio por regiao, a media de comodos, regioes  e comodos afetam os preços, qual tem mais impacto...?

tecnologia esperadas q serao usadas

* python
    * selenium ou beutiful soup para coleta de dados
    * pandas para exploraçao e limpeza

* power bi
    * para apresentaçao final

*github para fazer os historicos de evoluçao diaria e ter noçao de qto tempo levou para fazer o projeto (nao estou me obrigando a ter prazo, tenho outro trabalho entao sera feito nas horas disponiveis sem afetar outros compromissos)






In [13]:
#importando as bibliotecas necessarias
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import date

In [14]:
# explarando o site com filtro de casa para venda e na ordem de maiores valores
#driver_casa = webdriver.Chrome()
#driver_casa.get('https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda')
#driver_casa.maximize_window()
#original_window = driver_casa.current_window_handle

In [15]:
#pegando os anuncios das casas
#anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

In [16]:
#conferindo a quantidade de elementos 
#len(anuncios)

entrando no anuncio para verificar o que é possivel extrair 

fazendo uma breve verificacao em algums anuncio vi q os valores nao correspondem ao valor real,
 entao apos a coleta de dados sera necessario ainda uma limpeza 

 aqui ja da para extrair :

 valor do imovel
 
 condominio
 
 iptu
 
 area
 
 quantidade de banheiro
 
 quantidade de quartos
 
 quantidade de vagas na garagem
 
 rua, numero e bairro (cidade e estado sao sempre iguais, ja q é uma pesquisa de imoveis de sao jose dos campos)
 
 se esta por imobiliaria ou nao
 
 extrair tambem as informaçoes das descriçoes para complentar alguma informaçao
 
 a descricao tbm pode estar como caracteristica
 
 quantidade de andares
    

In [17]:
# window_handles eh uma lista de todas as guias q estao abertas, usar esse for pra saber quais os nomes das guias
#for window_handle in driver_casa.window_handles:
 #   print(window_handle)

In [18]:
#original_driver = driver_casa.current_window_handle
#original_driver

In [20]:
#url = driver_casa.current_url
#url = url.replace('pagina=1', 'pagina=2')
#print(url)

In [43]:
'''
#dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

for indice in range(121,161):
    driver_casa = webdriver.Chrome()
    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' 
    print(f'indice= {indice}')
    print()
    print(f'url = {url}')
    driver_casa.get(url)
    driver_casa.maximize_window()
    original_window = driver_casa.current_window_handle
    print(f'original_window == {original_window}')
    
    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

    for i in range(len(anuncios)):
        try:
            while len(driver_casa.window_handles) == 1: # fazer um break de emergencia
                print()
                print()
                print(f'indice == {i}')
                print(f'tamanho de len de driver_casa.window_handles == {len(driver_casa.window_handles)}')
                print(f'current window == {driver_casa.current_window_handle}')

                try:
                    print(driver_casa.current_window_handle)
                    print(f'indice == {i}')
                    driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click()
                    print(len(driver_casa.window_handles))
                except:
                    print('erro no click do anuncio')
                if len(driver_casa.window_handles) > 1 :
                    driver_casa.switch_to.window(driver_casa.window_handles[1])
                    break


            try:
                endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ') 
                print(f'Endereço = {endereco.text}')
            except:
                print('erro endereco =====================')

            try:
                preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')
                print(f'preco = {preco.text}')
            except:
                print('erro preco =====================')

            try:
                condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]
                print(f'condominio = {condominio.text}')
            except:
                print('erro condominio =====================')

            try:
                iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]
                print(f'iptu = {iptu.text}')
            except:
                print('erro iptu =====================')

            try:
                comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
                lista_comodos = [comodo.text for comodo in comodos]
                print(f'lista de comodos = {lista_comodos}')
            except:
                print('erro comodos =====================')

            try:
                caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
                lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]
                print(f'Caracteristica = {lista_caracteristicas}')
            except:
                print('erro caracteristicas =====================')

            try:
                obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
                print(f'obserevacoes = {obserevacoes.text}')
            except:
                print('erro obserevacoes =====================')

            try:
                url_id_anuncio = driver_casa.current_url
                print(f'url_id_anuncio = {url_id_anuncio}')
            except:
                print('erro url_id_anuncio =====================')
            

            #print()
            #print()
            #print(f'indice ===={i}')
            #print(driver_casa.current_window_handle)
                #print()
            #print(f'Endereço = {endereco.text}')
            #print(f'preco = {preco.text}')
            #print(f'condominio = {condominio.text}')
            #print(f'iptu = {iptu.text}')
            #print(f'Area = {lista_comodos[4]}')
            #print(f'Quartos = {lista_comodos[5]}')
            #print(f'Banheiros = {lista_comodos[-1]}')
            #print(f'Caracteristica = {caracteristica}')
            #print(f'obserevacoes = {obserevacoes.text}')
            #print()
            #print()


            caracteristica = [caracteristica for caracteristica in caracteristicas]
            dic_dados = {'Endereço':[endereco.text],
                     'Preço' : [preco.text],
                     'Condominio': [condominio.text],
                     'Iptu': [iptu.text],
                     'Lista Comodos': lista_comodos,
                     'Lista Caracteristicas' : lista_caracteristicas,
                     'Obserevacoes' : [obserevacoes.text],
                     'url_id_anuncio' : [url_id_anuncio]}


            while len(driver_casa.window_handles) > 1:
                driver_casa.switch_to.window(driver_casa.window_handles[1])
                print(driver_casa.current_window_handle)
                driver_casa.close()
                print(f'len(driver_casa.window_handles) ==== {len(driver_casa.window_handles)}')
                driver_casa.switch_to.window(original_window)
                print(driver_casa.current_window_handle)

            print()
            print()
            #print(f'indice ========== {i}')
            #print(dic_dados)
            #print()
            #print()
            dados = dados.append(dic_dados, ignore_index = True)
        except:
            print()
            print()
            print('************ FALHA de CARREGAR anuncio!!!!!!!!!!!!!!!!')
            print()
            print()
    driver_casa.quit()
dados'''

indice= 121

url = https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=121&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda
original_window == CDwindow-452EC58E4C03084A551B6473D693E8FA


indice == 0
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-452EC58E4C03084A551B6473D693E8FA
CDwindow-452EC58E4C03084A551B6473D693E8FA
indice == 0
2
Endereço = Monte Castelo, São José dos Campos - SP
preco = R$ 298.000
condominio = condomínio não informado
iptu = IPTU R$ 300
lista de comodos = ['', '', '', '', '68 m²', '2 quartos', '2 vagas', '1 banheiro']
Caracteristica = []
obserevacoes = Casa no MONTE CASTELO
2 dormitórios
1 banheiro armário e box
Sala
Cozinha
Área de serviço coberta
2 vagas de garagem coberta
Casa com estrutura para construção de outra casa em cima e possibilid

C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2326245658.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


2
Endereço = Jardim Santa Inês I, São José dos Campos - SP
preco = R$ 298.000
condominio = condomínio não informado
iptu = IPTU R$ 350
lista de comodos = ['', '', '', '', '87 m²', '3 quartos', '1 vaga', '2 banheiros']
Caracteristica = []
obserevacoes = Casa linda de 3 quartos
1 suíte
1 Banheiro comum
Sala
Cozinha
Área de serviço com churrasqueira
Garagem coberta
Armários na cozinha
Varanda
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-3-quartos-jardim-santa-ines-i-sao-jose-dos-campos-sp-87m2-id-2538817012/
CDwindow-BD634C6E3155F62DDD901775F210F8B5
len(driver_casa.window_handles) ==== 1
CDwindow-452EC58E4C03084A551B6473D693E8FA




indice == 2
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-452EC58E4C03084A551B6473D693E8FA
CDwindow-452EC58E4C03084A551B6473D693E8FA
indice == 2


C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2326245658.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


2
Endereço = Rua Luiz Borges Filho - Jardim Telespark, São José dos Campos - SP
preco = R$ 298.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '82 m²', '2 quartos', '1 vaga', '2 banheiros']
Caracteristica = []
obserevacoes = Casa com Edícula no Jd Telespark , Zona norte
Possui sala, cozinha com armário
- 2 dormitórios
-1 banheiro
Edícula possuí , sala , cozinha 1 dormitório 1 banheiro
Garagem 1 coberta
área de seviço
corredor coberta
Aceita financiamento -
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-2-quartos-jardim-telespark-sao-jose-dos-campos-sp-82m2-id-2503625832/
CDwindow-5CE7A5F9EB58503A63F1D2C9DA115042
len(driver_casa.window_handles) ==== 1
CDwindow-452EC58E4C03084A551B6473D693E8FA




C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2326245658.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados = dados.append(dic_dados, ignore_index = True)


,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"[Rua Coronel Manoel Martins Júnior, 0 - Jardim...",[R$ 42.000],[condomínio não informado],[IPTU não informado],"[, , , , 120 m², 2 quartos, 2 vagas, 1 banheiro]",[],[A casa está localizado no bairro Jardim Espla...,[https://www.zapimoveis.com.br/imovel/venda-ca...
1,"[Rio Comprido, São José dos Campos - SP]",[R$ 43.000],[condomínio não informado],[IPTU não informado],"[, , , , 70 m², 2 quartos, 2 vagas, 1 banheiro]",[],[Casa em construção termino da obra SET-2018!!...,[https://www.zapimoveis.com.br/imovel/venda-ca...
2,"[Rua Bariloche, 0 - Jardim América, São José d...",[R$ 43.000],[condomínio não informado],[IPTU não informado],"[, , , , 138 m², 3 quartos, 2 vagas, 2 banheiros]","[Aceita animais, Ar-condicionado, Churrasqueira]",[A casa está localizado no bairro Jardim Améri...,[https://www.zapimoveis.com.br/imovel/venda-ca...
3,"[Jardim Santa Júlia, São José dos Campos - SP]",[R$ 44.000],[condomínio não informado],[IPTU não informado],"[, , , , 62 m², 2 quartos, 1 vaga, 1 banheiro]","[Perto de vias de acesso, Próximo a escola, Pr...",[2 dormitórios wc social sala cozinha areá de ...,[https://www.zapimoveis.com.br/imovel/venda-ca...
4,"[Jardim Sao Jose II, São José dos Campos - SP]",[R$ 45.000],[condomínio não informado],[IPTU não informado],"[, , , 75 m², 1 quarto, 1 banheiro]",[],"[venda com urgencia ,casa se garagem e sem qui...",[https://www.zapimoveis.com.br/imovel/venda-ca...
...,...,...,...,...,...,...,...,...
3862,"Jardim Nova Michigan, São José dos Campos - SP",R$ 298.000,condomínio não informado,IPTU não informado,"[, , , , 67 m², 2 quartos, 2 vagas, 1 banheiro]","[Área de serviço, Cozinha, Lavanderia]",Casa a venda no jardim nova michigam\nPortão e...,https://www.zapimoveis.com.br/imovel/venda-cas...
3863,"Vila Rica, São José dos Campos - SP",R$ 298.000,condomínio não informado,IPTU não informado,"[, , , , 130 m², 3 quartos, 2 vagas, 2 banheiros]",[],"Casa residencial para Venda\nVila Rica, São Jo...",https://www.zapimoveis.com.br/imovel/venda-cas...
3864,"[Monte Castelo, São José dos Campos - SP]",[R$ 298.000],[condomínio não informado],[IPTU R$ 300],"[, , , , 68 m², 2 quartos, 2 vagas, 1 banheiro]",[],[Casa no MONTE CASTELO\n2 dormitórios\n1 banhe...,[https://www.zapimoveis.com.br/imovel/venda-ca...
3865,"[Jardim Santa Inês I, São José dos Campos - SP]",[R$ 298.000],[condomínio não informado],[IPTU R$ 350],"[, , , , 87 m², 3 quartos, 1 vaga, 2 banheiros]",[],[Casa linda de 3 quartos\n1 suíte\n1 Banheiro ...,[https://www.zapimoveis.com.br/imovel/venda-ca...


In [42]:
dados2

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes


In [38]:
dados

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"[Rua Coronel Manoel Martins Júnior, 0 - Jardim...",[R$ 42.000],[condomínio não informado],[IPTU não informado],"[, , , , 120 m², 2 quartos, 2 vagas, 1 banheiro]",[],[A casa está localizado no bairro Jardim Espla...,[https://www.zapimoveis.com.br/imovel/venda-ca...
1,"[Rio Comprido, São José dos Campos - SP]",[R$ 43.000],[condomínio não informado],[IPTU não informado],"[, , , , 70 m², 2 quartos, 2 vagas, 1 banheiro]",[],[Casa em construção termino da obra SET-2018!!...,[https://www.zapimoveis.com.br/imovel/venda-ca...
2,"[Rua Bariloche, 0 - Jardim América, São José d...",[R$ 43.000],[condomínio não informado],[IPTU não informado],"[, , , , 138 m², 3 quartos, 2 vagas, 2 banheiros]","[Aceita animais, Ar-condicionado, Churrasqueira]",[A casa está localizado no bairro Jardim Améri...,[https://www.zapimoveis.com.br/imovel/venda-ca...
3,"[Jardim Santa Júlia, São José dos Campos - SP]",[R$ 44.000],[condomínio não informado],[IPTU não informado],"[, , , , 62 m², 2 quartos, 1 vaga, 1 banheiro]","[Perto de vias de acesso, Próximo a escola, Pr...",[2 dormitórios wc social sala cozinha areá de ...,[https://www.zapimoveis.com.br/imovel/venda-ca...
4,"[Jardim Sao Jose II, São José dos Campos - SP]",[R$ 45.000],[condomínio não informado],[IPTU não informado],"[, , , 75 m², 1 quarto, 1 banheiro]",[],"[venda com urgencia ,casa se garagem e sem qui...",[https://www.zapimoveis.com.br/imovel/venda-ca...
...,...,...,...,...,...,...,...,...
3835,"[Rua Doutor Raphael Faro Netto, 131 - Parque I...",[R$ 320.000],[condomínio não informado],[IPTU não informado],"[, , , , 120 m², 3 quartos, 2 vagas, 2 banheiros]",[],[Casa\nSala para 2 ambientes\n03 Dormitórios\n...,[https://www.zapimoveis.com.br/imovel/venda-ca...
3836,"[Vila Maria, São José dos Campos - SP]",[R$ 320.000],[condomínio não informado],[IPTU não informado],"[, , , , 109 m², 3 quartos, 2 vagas, 2 banheiros]","[Área de serviço, Cozinha, Lavanderia]","[Casa na Vila Maria a 3 min do Centro, 3 dormi...",[https://www.zapimoveis.com.br/imovel/venda-ca...
3837,"[Conjunto Residencial Dom Pedro I, São José do...",[R$ 320.000],[condomínio não informado],[IPTU não informado],"[, , , , 160 m², 8 quartos, 2 vagas, 2 banheiros]","[Área de serviço, Cozinha, Lavanderia, Varanda]","[Ótimo Sobrado, Zona Sul, Bairro D. Pedro I , ...",[https://www.zapimoveis.com.br/imovel/venda-ca...
3838,"[Rua Tereza de Jesus Silva, 476 - Parque Inter...",[R$ 320.000],[condomínio não informado],[IPTU não informado],"[, , , , 135 m², 3 quartos, 2 vagas, 2 banheiros]",[],"[casa são 3 quartos, dois na parte de cima (os...",[https://www.zapimoveis.com.br/imovel/venda-ca...


In [39]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
tabela_imoveis_raw = dados.to_csv(f'tabela_imoveis_raw_{data_atual}.csv', sep = ',', index = False)

In [40]:
df = pd.read_csv('tabela_imoveis_raw_12_3_2022.csv')
df

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"['Rua Coronel Manoel Martins Júnior, 0 - Jardi...",['R$ 42.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '120 m²', '2 quartos', '2 vag...",[],['A casa está localizado no bairro Jardim Espl...,['https://www.zapimoveis.com.br/imovel/venda-c...
1,"['Rio Comprido, São José dos Campos - SP']",['R$ 43.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '70 m²', '2 quartos', '2 vaga...",[],['Casa em construção termino da obra SET-2018!...,['https://www.zapimoveis.com.br/imovel/venda-c...
2,"['Rua Bariloche, 0 - Jardim América, São José ...",['R$ 43.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '138 m²', '3 quartos', '2 vag...","['Aceita animais', 'Ar-condicionado', 'Churras...",['A casa está localizado no bairro Jardim Amér...,['https://www.zapimoveis.com.br/imovel/venda-c...
3,"['Jardim Santa Júlia, São José dos Campos - SP']",['R$ 44.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '62 m²', '2 quartos', '1 vaga...","['Perto de vias de acesso', 'Próximo a escola'...",['2 dormitórios wc social sala cozinha areá de...,['https://www.zapimoveis.com.br/imovel/venda-c...
4,"['Jardim Sao Jose II, São José dos Campos - SP']",['R$ 45.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '75 m²', '1 quarto', '1 banheiro']",[],"['venda com urgencia ,casa se garagem e sem qu...",['https://www.zapimoveis.com.br/imovel/venda-c...
...,...,...,...,...,...,...,...,...
3835,"['Rua Doutor Raphael Faro Netto, 131 - Parque ...",['R$ 320.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '120 m²', '3 quartos', '2 vag...",[],['Casa\nSala para 2 ambientes\n03 Dormitórios\...,['https://www.zapimoveis.com.br/imovel/venda-c...
3836,"['Vila Maria, São José dos Campos - SP']",['R$ 320.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '109 m²', '3 quartos', '2 vag...","['Área de serviço', 'Cozinha', 'Lavanderia']","['Casa na Vila Maria a 3 min do Centro, 3 dorm...",['https://www.zapimoveis.com.br/imovel/venda-c...
3837,"['Conjunto Residencial Dom Pedro I, São José d...",['R$ 320.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '160 m²', '8 quartos', '2 vag...","['Área de serviço', 'Cozinha', 'Lavanderia', '...","['Ótimo Sobrado, Zona Sul, Bairro D. Pedro I ,...",['https://www.zapimoveis.com.br/imovel/venda-c...
3838,"['Rua Tereza de Jesus Silva, 476 - Parque Inte...",['R$ 320.000'],['condomínio não informado'],['IPTU não informado'],"['', '', '', '', '135 m²', '3 quartos', '2 vag...",[],"['casa são 3 quartos, dois na parte de cima (o...",['https://www.zapimoveis.com.br/imovel/venda-c...


In [ ]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
data_atual

In [54]:
dados2 = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

for indice in range(161,201):
    driver_casa = webdriver.Chrome()
    url = f'https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina={indice}&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda' 
    print(f'indice= {indice}')
    print()
    print(f'url = {url}')
    driver_casa.get(url)
    driver_casa.maximize_window()
    original_window = driver_casa.current_window_handle
    print(f'original_window == {original_window}')
    
    anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

    for i in range(len(anuncios)):
        try:
            while len(driver_casa.window_handles) == 1: # fazer um break de emergencia
                print()
                print()
                print(f'indice == {i}')
                print(f'tamanho de len de driver_casa.window_handles == {len(driver_casa.window_handles)}')
                print(f'current window == {driver_casa.current_window_handle}')

                try:
                    print(driver_casa.current_window_handle)
                    print(f'indice == {i}')
                    driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click()
                    print(len(driver_casa.window_handles))
                except:
                    print('erro no click do anuncio')
                if len(driver_casa.window_handles) > 1 :
                    driver_casa.switch_to.window(driver_casa.window_handles[1])
                    break


            try:
                endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ') 
                print(f'Endereço = {endereco.text}')
            except:
                print('erro endereco =====================')

            try:
                preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')
                print(f'preco = {preco.text}')
            except:
                print('erro preco =====================')

            try:
                condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]
                print(f'condominio = {condominio.text}')
            except:
                print('erro condominio =====================')

            try:
                iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]
                print(f'iptu = {iptu.text}')
            except:
                print('erro iptu =====================')

            try:
                comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
                lista_comodos = [comodo.text for comodo in comodos]
                print(f'lista de comodos = {lista_comodos}')
            except:
                print('erro comodos =====================')

            try:
                caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
                lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]
                print(f'Caracteristica = {lista_caracteristicas}')
            except:
                print('erro caracteristicas =====================')

            try:
                obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
                print(f'obserevacoes = {obserevacoes.text}')
            except:
                print('erro obserevacoes =====================')

            try:
                url_id_anuncio = driver_casa.current_url
                print(f'url_id_anuncio = {url_id_anuncio}')
            except:
                print('erro url_id_anuncio =====================')
            

            #print()
            #print()
            #print(f'indice ===={i}')
            #print(driver_casa.current_window_handle)
                #print()
            #print(f'Endereço = {endereco.text}')
            #print(f'preco = {preco.text}')
            #print(f'condominio = {condominio.text}')
            #print(f'iptu = {iptu.text}')
            #print(f'Area = {lista_comodos[4]}')
            #print(f'Quartos = {lista_comodos[5]}')
            #print(f'Banheiros = {lista_comodos[-1]}')
            #print(f'Caracteristica = {caracteristica}')
            #print(f'obserevacoes = {obserevacoes.text}')
            #print()
            #print()


            caracteristica = [caracteristica for caracteristica in caracteristicas]
            dic_dados = {'Endereço':endereco.text,
                     'Preço' : preco.text,
                     'Condominio': condominio.text,
                     'Iptu': iptu.text,
                     'Lista Comodos': lista_comodos,
                     'Lista Caracteristicas' : lista_caracteristicas,
                     'Obserevacoes' : obserevacoes.text,
                     'url_id_anuncio' : url_id_anuncio}


            while len(driver_casa.window_handles) > 1:
                driver_casa.switch_to.window(driver_casa.window_handles[1])
                print(driver_casa.current_window_handle)
                driver_casa.close()
                print(f'len(driver_casa.window_handles) ==== {len(driver_casa.window_handles)}')
                driver_casa.switch_to.window(original_window)
                print(driver_casa.current_window_handle)

            print()
            print()
            #print(f'indice ========== {i}')
            #print(dic_dados)
            #print()
            #print()
            dados2 = dados2.append(dic_dados, ignore_index = True)
        except:
            print()
            print()
            print('************ FALHA de CARREGAR anuncio!!!!!!!!!!!!!!!!')
            print()
            print()
    driver_casa.quit()
dados2

indice= 161

url = https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=161&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda
original_window == CDwindow-34D4DF34FC6F5C999F3C853B0374E799


indice == 0
tamanho de len de driver_casa.window_handles == 1
current window == CDwindow-34D4DF34FC6F5C999F3C853B0374E799
CDwindow-34D4DF34FC6F5C999F3C853B0374E799
indice == 0
2
Endereço = Rua Osvaldo da Graça Almeida Júnior, 0 - Jardim Santa Júlia, São José dos Campos - SP
preco = R$ 320.000
condominio = condomínio não informado
iptu = IPTU R$ 300
lista de comodos = ['', '', '', '', '116 m²', '3 quartos', '2 vagas', '1 banheiro']
Caracteristica = []
obserevacoes = Essa casa está localizado no Jardim Santa Júlia, possui fácil acesso as principais vias da cidade.

Conheça as características desta linda casa.

-

C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2381365105.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


2
Endereço = Rua Galgos, 0 - Jardim Satélite, São José dos Campos - SP
preco = R$ 320.000
condominio = condomínio não informado
iptu = IPTU R$ 600
lista de comodos = ['', '', '', '', '150 m²', '2 quartos', '2 vagas', '1 banheiro']
Caracteristica = []
obserevacoes = CASA TÉRREA NO JD.SATÉLITE

*TODA REFORMADA: Piso novo, Hidraulica, Encanamento, Pintura ....tudo novo.

-3 dormitórios
-Sala
-Cozinha
-Banheiro
-Cozinha
-Área de serviço
-2 vagas de garagem
-10 mts de frente

-Próximo ao Poliesportivo João do Pulo
-Acesso fácil para Shopping Vale sul, Dutra

Entre em contato e agende já sua visita!

Referência: 31592

Felipe Andrade Gusmão
Corretor de Imóveis
Creci: 178866
Mostrar mais
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-2-quartos-jardim-satelite-sao-jose-dos-campos-sp-150m2-id-2543743926/
CDwindow-A40BEAE30106EE71D76731E42B9E801D
len(driver_casa.window_handles) ==== 1
CDwindow-34D4DF34FC6F5C999F3C853B0374E799




indice == 2
tamanho de len de driver_casa.window

C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2381365105.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


2
Endereço = Vila São Pedro, São José dos Campos - SP
preco = R$ 320.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '100 m²', '3 quartos', '1 vaga', '2 banheiros']
Caracteristica = ['Churrasqueira', 'Cozinha', 'Garagem']
obserevacoes = Ótima casa localizado no Bairro Vila São Pedro ;
São 125 m² de terreno e 100 m² de construção ;
3 dormitórios , 2 banheiros ;
Cozinha Ampla ,
Garagem coberta 1 carro ;
Portão automático .
Terraço com churrasqueira ;
Estuda permuta por casa térrea na região ou apartamento com elevador.

Características do imóvel
Área de serviço
Churrasqueira
Cozinha
Piso frio
Sala de estar

Mais informações em www.movaeimoveis.com.br
Mostrar mais
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-3-quartos-com-churrasqueira-vila-sao-pedro-sao-jose-dos-campos-sp-100m2-id-2530050390/
CDwindow-6B0CB0862C858EBAAC06BA79A41E333E
len(driver_casa.window_handles) ==== 1
CDwindow-34D4DF34FC6F5C999F3C853B0374E799

...

C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2381365105.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


2
Endereço = Jardim Santa Inês I, São José dos Campos - SP
preco = R$ 340.000
condominio = condomínio não informado
iptu = IPTU não informado
lista de comodos = ['', '', '', '', '160 m²', '1 quarto', '8 vagas', '1 banheiro']
Caracteristica = ['Área de serviço', 'Churrasqueira', 'Jardim', 'Varanda']
obserevacoes = VARANDA, AREA GOURMET COM FOGÃO A LENHA, COMDOOS AMPLOS, SOMENTE A VISTA REF 63345 - R$ R$ 340.000,00 - Maciel Negócios Imobiliários Ltda - CRECI 11.956 J - www.imobiliariamaciel.com.br Atualizado em
url_id_anuncio = https://www.zapimoveis.com.br/imovel/venda-casa-1-quarto-com-churrasqueira-jardim-santa-ines-i-sao-jose-dos-campos-sp-160m2-id-2493599823/
CDwindow-67799BD7610873BDB36AA8AAE5AC64FF
len(driver_casa.window_handles) ==== 1
CDwindow-C07BA70577DEDB4A1EFB29D6941C241A




C:\Users\jpnun\AppData\Local\Temp\ipykernel_7192\2381365105.py:131: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados2 = dados2.append(dic_dados, ignore_index = True)


indice= 179

url = https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=179&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda


WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x009869A3+2582947]
	Ordinal0 [0x0091A6D1+2139857]
	Ordinal0 [0x00813A98+1063576]
	Ordinal0 [0x00810551+1049937]
	Ordinal0 [0x00805D9D+1007005]
	Ordinal0 [0x00806A0E+1010190]
	Ordinal0 [0x00805FFE+1007614]
	Ordinal0 [0x008055F0+1005040]
	Ordinal0 [0x00804727+1001255]
	Ordinal0 [0x008049E6+1001958]
	Ordinal0 [0x0081518A+1069450]
	Ordinal0 [0x008688FD+1411325]
	Ordinal0 [0x0085854C+1344844]
	Ordinal0 [0x0086834A+1409866]
	Ordinal0 [0x00858366+1344358]
	Ordinal0 [0x00835176+1200502]
	Ordinal0 [0x00836066+1204326]
	GetHandleVerifier [0x00B2BE02+1675858]
	GetHandleVerifier [0x00BE036C+2414524]
	GetHandleVerifier [0x00A1BB01+560977]
	GetHandleVerifier [0x00A1A8D3+556323]
	Ordinal0 [0x0092020E+2163214]
	Ordinal0 [0x00925078+2183288]
	Ordinal0 [0x009251C0+2183616]
	Ordinal0 [0x0092EE1C+2223644]
	BaseThreadInitThunk [0x7627FA29+25]
	RtlGetAppContaine

In [55]:
dados2

,Endereço,Preço,Condominio,Iptu,Lista Comodos,Lista Caracteristicas,Obserevacoes,url_id_anuncio
0,"Rua Osvaldo da Graça Almeida Júnior, 0 - Jardi...",R$ 320.000,condomínio não informado,IPTU R$ 300,"[, , , , 116 m², 3 quartos, 2 vagas, 1 banheiro]",[],Essa casa está localizado no Jardim Santa Júli...,https://www.zapimoveis.com.br/imovel/venda-cas...
1,"Rua Galgos, 0 - Jardim Satélite, São José dos ...",R$ 320.000,condomínio não informado,IPTU R$ 600,"[, , , , 150 m², 2 quartos, 2 vagas, 1 banheiro]",[],CASA TÉRREA NO JD.SATÉLITE\n\n*TODA REFORMADA:...,https://www.zapimoveis.com.br/imovel/venda-cas...
2,"Vila São Pedro, São José dos Campos - SP",R$ 320.000,condomínio não informado,IPTU não informado,"[, , , , 100 m², 3 quartos, 1 vaga, 2 banheiros]","[Churrasqueira, Cozinha, Garagem]",Ótima casa localizado no Bairro Vila São Pedro...,https://www.zapimoveis.com.br/imovel/venda-cas...
3,Rua Antonio Marcos de Oliveira - Jardim Marian...,R$ 320.000,condomínio não informado,IPTU R$ 320,"[, , , , 144 m², 3 quartos, 2 vagas, 2 banheiros]","[Área de serviço, Cozinha, Quintal, Varanda]",Sobrado a venda do Jd Mariana II\n\n03 Dormitó...,https://www.zapimoveis.com.br/imovel/venda-cas...
4,"Rua Abaré, 1073 - Jardim Vale do Sol, São José...",R$ 320.000,condomínio não informado,IPTU não informado,"[, , , , 74 m², 3 quartos, 1 vaga, 1 banheiro]",[],A casa de vila sobrado está localizado no bair...,https://www.zapimoveis.com.br/imovel/venda-cas...
...,...,...,...,...,...,...,...,...
427,"Residencial União, São José dos Campos - SP",R$ 340.000,condomínio não informado,IPTU não informado,"[, , , , 161 m², 3 quartos, 2 vagas, 3 banheiros]","[Área de serviço, Churrasqueira, Cozinha, Quin...",Ótimo sobrado.\nNa zona sul.\nCom edícula.\nCh...,https://www.zapimoveis.com.br/imovel/venda-cas...
428,"Cidade Vista Verde, São José dos Campos - SP",R$ 340.000,condomínio não informado,IPTU não informado,"[, , , , 62 m², 2 quartos, 2 vagas, 1 banheiro]","[Área de serviço, Varanda]","CASA A VENDA, CIDADE VISTA VERDE, SÃO JOSÉ DOS...",https://www.zapimoveis.com.br/imovel/venda-cas...
429,Rua Doutor Raphael Faro Netto - Parque Interla...,R$ 340.000,condomínio não informado,IPTU R$ 270,"[, , , , 120 m², 3 quartos, 2 vagas, 2 banheiros]",[],Vende-se casa recém reformada\nPiso porcelanat...,https://www.zapimoveis.com.br/imovel/venda-cas...
430,"Parque Industrial, São José dos Campos - SP",R$ 340.000,condomínio não informado,IPTU não informado,"[, , , , 100 m², 2 quartos, 1 vaga, 2 banheiros]","[Área de serviço, Varanda]","REF 75784 - R$ R$ 340.000,00 - Maciel Negócios...",https://www.zapimoveis.com.br/imovel/venda-cas...


In [57]:
data_atual = date.today()
data_atual = f'{data_atual.day}_{data_atual.month}_{data_atual.year}'
tabela_imoveis_raw = dados2.to_csv(f'tabela_imoveis_new_raw_{data_atual}.csv', sep = ',', index = False)
